In [1]:
import sys, os, glob
sys.path.append('../')

import tensorflow as tf
import numpy as np

from MRIsegm.utils import get_slices
from MRIsegm.processing import pre_processing_data, predict_images, contour_slices
from MRIsegm.metrics import dice_coef
from MRIsegm.losses import DiceBCEloss
from segmentation_models.losses import DiceLoss, BinaryFocalLoss

from PIL import Image

Segmentation Models: using `keras` framework.


In [2]:
models_dir = '../data/models/'
model_name = 'efficientnetb0_BTC=4_full_150E_OPT=adam_LOSS=dice_loss_plus_1binary_focal_loss'
model_path = os.path.join(models_dir, model_name + '.h5')

dice_loss = DiceLoss()
focal_loss = BinaryFocalLoss()
loss = dice_loss + (1 * focal_loss)

dependencies = {
    'DiceBCEloss': DiceBCEloss,
    'dice_coef': dice_coef,
    'FixedDropout': tf.keras.layers.Dropout(0.1),
    'dice_loss_plus_1binary_focal_loss': loss
}

model = tf.keras.models.load_model(model_path, custom_objects=dependencies)

In [3]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patients = os.listdir(src)
if '.DS_Store' in patients:
    patients.remove('.DS_Store')
print(f"Number of patients: {len(patients)}")

Number of patients: 48


In [5]:
unwanted = ['ROI', '_frames', 'bis', '_predicted', '_resized', '_contoured', '_3D', '_NRRD']

for patient in patients:

    print(f'{patient}')
    try:
        folders=glob.glob(src + '/' + patient + '/T2*')
        folders = [x for x in folders if all(y not in x for y in unwanted)]
        assert len(folders) != 0
    except:
        folders=glob.glob(src + '/' + patient + '/t2*')
        folders = [x for x in folders if all(y not in x for y in unwanted)]



    for path in folders:
        folder_name = os.path.split(path)[1]
        print(f' folder: {folder_name}')
        slices = get_slices(dir_path=path, uint8=False)
        preprocessed = pre_processing_data(slices)

        predicted = predict_images(preprocessed, model)
        predicted_sq = np.squeeze(predicted, axis=-1)


        predicted_folder = '_'.join((path, 'predicted_mask'))
        if not os.path.exists(predicted_folder):
            os.makedirs(predicted_folder)
   
        pred = predicted_sq
        for i in range(pred.shape[0]):
            pred[i, ...] = np.where(pred[i, ...] >= 0.1, 1, 0)
            img = Image.fromarray(pred[i, ...] * 255).convert('L')
            filename = str(i) + '.png'
            output =  os.path.join(predicted_folder, filename)
            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)
    

        contoured = contour_slices(preprocessed, predicted)

        contoured_folder = '_'.join((path, 'contoured'))
        if not os.path.exists(contoured_folder):
            os.makedirs(contoured_folder)


        for z in range(contoured.shape[0]):

            img = Image.fromarray(contoured[z, ...], mode='RGB')
            filename = str(z) + '.png'
            output =  os.path.join(contoured_folder, filename)

            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)

BO77
 folder: T2
BO48
 folder: T2
BO47
 folder: T2
BO78
 folder: T2
BO40
 folder: T2AX
BO76
 folder: T2
BO82
 folder: T2
BO49
 folder: T2
BO85
 folder: T2
BO71
 folder: T2
BO2
 folder: T2AX
BO54
 folder: T2
BO31
 folder: t2DEF
BO36
 folder: T2AX
BO38
 folder: T2AXAlta
 folder: T2AXbassa
BO64
 folder: T2
BO90
 folder: T25mm
BO63
 folder: T2
BO52
 folder: T2
BO39
 folder: T2AX
BO37
 folder: T2AX
BO74
 folder: T2
BO45
 folder: T2
BO42
 folder: T2AX
BO29
 folder: T2AX
BO16
 folder: T2AX
BO11
 folder: T2AX
BO18
 folder: T2AX
BO43
 folder: T2AX
BO44
 folder: T2AX
BO86
 folder: T2
BO72
 folder: T2
BO75
 folder: T2
BO26
 folder: T2AX
BO28
 folder: T2AX
BO17
 folder: T2AX
BO9
 folder: T2AX
BO1
 folder: T2AX
BO35
 folder: T2AX
BO32
 folder: T2AX
BO68
 folder: T2
BO50
 folder: T2
BO66
 folder: T2
BO61
 folder: T2
BO33
 folder: T2AX
BO60
 folder: T2
BO51
 folder: T2
BO56
 folder: T25mm
 folder: T2
